# Introduction/Business Problem

A European-based rock climbing gear manufacturer is planning the launch of a brand new climbing shoe that they consider 'revolutionary'.  Backed by the slogan of "Send Anything; Send Always", the company is claiming that the new shoes will enable climbers of all skill levels to climb (aka "Send") harder routes than they have ever climbed in the past due to a new rubber compound they have formulated. 

The company's marketing campaign will be critical to driving interest in the new shoes, and their digital marketing team has come up with their vision for the content they will produce for online ads.  The team has decided that they want to photograph their best sponsored athlete on location, in the United States, climbing some of the most technically difficult rock climbs the US has to offer - showcasing the new shoes' high performance capabilities.

The team they will assemble has been selected; however, due to upstream delays in production, their window for capturing their marketing content has been compressed.  The team will need to schedule their travel fairly quickly, and they need to determine where they will be travelling to within the United States.  This will be the first time to the United States for the entire team, so they need help determining the best location.  There are requirements for where they will travel:

- There needs to be a high volume of technically difficult climbing routes in the area; they need to be hard enough to highlight the shoes' performance
- Because this is the team's first time to the US, they want to be able to enjoy themselves after working all day in the mountains.  The team has a 'work hard; play hard' mentality, so they are looking to stay in a town that offers a lot of options for pubs and breweries

The team has only heard of a few potential locations that are of interest (e.g. Yosemite National Park; Red Rocks near Las Vegas; "The Gunks" in New York); however, they don't have the time themselves to do extensive research into multiple locations.  They have asked a Data Scientist to quickly access different data sources and to perform some fundamental analysis to identify their final destination.  Ultimately, the team needs a recommendation for where they should narrow their search when booking their Air BnB.

# Data Sources

#### Data Source #1: Mountain Project:

Mountain Project is a crowd-sourced platform for Rock Climbers to catalogue and inventory information about specific rock climbing routes and areas.  It is largely considered the definitive source for rock climbing route information.  While separate guide books are often created for individual climbing areas to provide even greater details about the climbs, Mountain Project (MP) is the preeminent, centralized platform for climbing information.  I will leverage Mountain Project's API to pull data from their databases.

A basic primer on rock climbing route information:

    - All routes are named by the individual who is the first ascensionist
    - All routes are given a grade/rating for their difficulty; this is a consensus-based system where the grade is determined based on the opinions of all individuals who climb the route
    - Mountain Project allows its user community to rate the climb for enjoyment factor, which is captured in a 0-4 star format
    - Mountain Project retains latitude and longitude data for each climb
    - Mountain Project also allows its user community to leave commentary, upload pictures, start topic threads, etc.  Note:  This data will likely not be leveraged during this assignment
    
Additional information on the system for rock climbing grading difficulty:

    - All routes are given a grade/rating for their difficulty; this is a consensus-based system where the grade is determined based on the opinions of all individuals who climb the route
    - The grading system leveraged is known as the Yosemite Decimal System, which is a number-based scale to signal difficulty
    - The numbers start at 5.0 (which is the easiest rock climbing grade) and incrementally increase by 0.1 as difficulty increases.  So, the next level after 5.0 is 5.1, then 5.2, 5.3 ... etc
    - The pattern for the grading convention shifts slightly after 5.9; grades then add 0.01 and one of 4 letters (a, b, c, d) to indicate difficulty.  So, the next level after 5.9 is 5.10a, then 5.10b, 5.10c, 5.10d, 5.11a, 5.11b ... etc.  
    - The current max difficulty grade is 5.15d
    - Pro level rock climbing is generally considered to start at 5.14a

Because a requirement from the marketing team is to find an area with a high density of difficult climbing for their sponsored athlete to climb, this analysis will define a "difficult climb" as anything graded 5.14a or higher.  This parameter will be passed through the API call to MP during the assignment.

Mountain Project Website:
https://www.mountainproject.com/

#### Data Source #2: FourSquare:


###### Note:  Because this capstone, as well as required assignments in the course, mandates the use of FourSquare and its API, I am going to assume this is familiar to the reader/audience.  I will, therefore, make the description of this data source more concise than the Mountain Project description.

FourSquare is a crowd-sourced, location-based service that offers data about venues around the world.  According to FourSquare's website: "Our global point-of-interest (POI) data, rich with context, is based on over 46K trusted sources and validated by millions of consumers. Create location-based experiences with a map unlike any other."  Venue data includes, but is not limited to: Name, category, rating, latitude, longitude, etc.  

This analysis will leverage the FourSquare API to analyze information pertaining to the Marketing team's second requirement: finding a town that offers a high concentration of, and ample options for, bars, breweries, gastro pubs, etc.


FourSquare Website:  https://foursquare.com/
